# MagicArticulate API Server for Google Colab

这个 notebook 将在 Colab 中启动一个 API 服务器，使你的本地 ArticulateHub 后端能够使用 GPU 加速的 MagicArticulate 模型。

## 1. 检查 GPU 可用性

In [ ]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. 克隆 MagicArticulate 仓库

In [ ]:
!git clone https://github.com/your-repo/MagicArticulate.git
%cd MagicArticulate

## 3. 安装依赖

In [ ]:
# 安装 MagicArticulate 依赖
!pip install -r requirements.txt

# 安装 API 服务器依赖
!pip install fastapi uvicorn pyngrok nest-asyncio python-multipart

## 4. 设置 Ngrok 认证（可选）

In [ ]:
# 如果你有 ngrok 账号，可以设置 authtoken 以获得更稳定的连接
# 从 https://dashboard.ngrok.com/get-started/your-authtoken 获取你的 token
# !ngrok authtoken YOUR_AUTH_TOKEN

## 5. 创建并启动 API 服务器

In [ ]:
import os
import sys
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import shutil
import tempfile
import json

# 允许在 Jupyter 环境中运行 asyncio
nest_asyncio.apply()

# 添加 MagicArticulate 到 Python 路径
sys.path.append('/content/MagicArticulate')

# 导入 MagicArticulate（根据实际的模块名调整）
# from magicarticulate import MagicArticulate

# 创建 FastAPI 应用
app = FastAPI(title="MagicArticulate API Server")

# 配置 CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 创建结果目录
RESULTS_DIR = "/content/results"
os.makedirs(RESULTS_DIR, exist_ok=True)

@app.get("/")
async def root():
    return {"message": "MagicArticulate API Server is running on Colab!"}

@app.post("/process")
async def process_model(
    file: UploadFile = File(...),
    model_id: str = Form(...),
    user_prompt: str = Form(...),
    template_id: str = Form(...),
    prompt_weight: float = Form(...)
):
    """
    处理上传的3D模型
    """
    temp_dir = tempfile.mkdtemp()
    try:
        # 保存上传的文件
        input_path = os.path.join(temp_dir, file.filename)
        with open(input_path, "wb") as f:
            shutil.copyfileobj(file.file, f)
        
        # 准备输出路径
        output_filename = f"{model_id}_articulated.obj"
        output_path = os.path.join(RESULTS_DIR, output_filename)
        
        print(f"Processing model: {file.filename}")
        print(f"Model ID: {model_id}")
        print(f"User prompt: {user_prompt}")
        print(f"Template: {template_id}, Weight: {prompt_weight}")
        
        # TODO: 在这里调用实际的 MagicArticulate 处理
        # result = magic_articulate.process(
        #     input_path=input_path,
        #     output_path=output_path,
        #     prompt=user_prompt,
        #     template=template_id,
        #     weight=prompt_weight
        # )
        
        # 模拟处理（实际使用时删除这部分）
        shutil.copy(input_path, output_path)
        
        return {
            "result_path": output_path,
            "result_filename": output_filename,
            "message": "Model processed successfully"
        }
        
    except Exception as e:
        print(f"Error processing model: {e}")
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        # 清理临时文件
        shutil.rmtree(temp_dir, ignore_errors=True)

@app.get("/download/{filename}")
async def download_result(filename: str):
    """
    下载处理结果
    """
    file_path = os.path.join(RESULTS_DIR, filename)
    if not os.path.exists(file_path):
        raise HTTPException(status_code=404, detail="File not found")
    
    return FileResponse(file_path, filename=filename)

@app.get("/status")
async def get_status():
    """
    获取服务器状态
    """
    return {
        "status": "running",
        "gpu_available": torch.cuda.is_available(),
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None
    }

# 启动服务器
ngrok_tunnel = ngrok.connect(8000)
print(f"\n" + "="*50)
print(f"🚀 API Server is running!")
print(f"Public URL: {ngrok_tunnel.public_url}")
print(f"\n请将这个 URL 配置到你的本地 backend 的环境变量中:")
print(f"AI_SERVICE_URL={ngrok_tunnel.public_url}")
print("="*50 + "\n")

# 启动 FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)